In [16]:
import os
import openai

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate(prompt,max_tokens=4000, temperature=0):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=temperature,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]['text']

In [2]:
import json

test_set = json.load(open('data/iirc_reranked.json','r'))

In [7]:
dev = json.load(open("./data/explained_dev.json",'r'))

In [4]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')

2022-10-21 11:47:58.298767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-21 11:47:58.298809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
passages = []
for q in dev:
    text = "{0}".format(q['question'])
    passages.append(text)
passages_embeddings = model.encode(passages, show_progress_bar=True, batch_size=128)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [14]:
import numpy as np
import torch
from tqdm import tqdm
import random

RANDOM_PROMPT = False
KSHOT = 4
USE_STATIC_PROMPT = False
STATIC_PROMPT = "For each example, use the documents to provide an answer to the question and cite evidence from the documents to support the answer. If there is not enough information in the documents to answer the question, then state "not enough information".\n\nExample 1:\n\nDocument 1: Title: Fred J. Shields. Content: Ollie Murphy\'s first-half goal gave \'the Royals\' a huge boost at half-time.\n\nDocument 2: Title: Ollie Murphy. Content: He plays club football for Carnaross\n\nDocument 3: Title: Ollie Murphy. Content: He came to national prominence in 1999 when he was one of Meath\'s best player\'s\n\nQuestion: Based on the above documents, Did Ollie Murphy play for any teams other than \'the Royals\'?\n\nEvidence: Document 1 says that Ollie Murphy\'s first-half goal gave \'the Royals\' a huge boost at half-time. However, this does not necessarily mean that Ollie only played for \'the Royals\'. Document 2 states that Ollie plays club football for Carnaross. This suggests that Ollie may have played for other teams in addition to \'the Royals\'. Document 3 says that Ollie came to national prominence in 1999 when he was one of Meath\'s best players. This also suggests that Ollie has played for other teams. Therefore, it is likely that Ollie has played for teams other than \'the Royals\'.\n\nAnswer: yes.\n\nExample 2:\n\nDocument 1: Title: Don Rendell. Content: The club played in the Brunei Premier League in the early 2000s, winning the league title in 2002 and 2004.\n\nQuestion: Based on the above documents, What club came in second at the 2004 Brunei Premier League?\n\nEvidence:  There is not enough information in the documents to answer the question.\n\nAnswer: not enough information.\n\nExample 3:\n\nDocument 1: Title: Stacy Compton. Content: Despicable Me, the first film in the series, and the first film from Illumination, was released on July 9, 2010.\n\nDocument 2: Title: Miranda Cosgrove. Content: Cosgrove\'s first television appearance (aside from commercials) was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville.\n\nDocument 3: Title: Miranda Cosgrove. Content: In 2004, Cosgrove soon landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh\n\nDocument 4: Title: Miranda Cosgrove. Content: Also in 2004, Cosgrove guest-starred in a special episode of the animated series What\'s New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life\n\nDocument 5: Title: Miranda Cosgrove. Content: The television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch\n\nDocument 6: Title: Miranda Cosgrove. Content: The first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous,\n\nDocument 7: Title: Miranda Cosgrove. Content: However, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007.\n\nQuestion: Based on the above documents, How many TV shows had Miranda Cosgrove been featured in by the year Despicable Me was released?\n\nEvidence: According to document 1, Despicable Me was released on July 9, 2010.Document 2 states that Cosgrove\'s first television appearance was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville.Document 3 says that, in 2004, Cosgrove landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh.Document 4 states that, also in 2004, Cosgrove guest-starred in a special episode of the animated series What\'s New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life.Document 5 says that the television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch.Document 6 states that the first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous.Document 7 says that, however, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007.Therefore, Miranda Cosgrove had been featured in 8 TV shows by the year Despicable Me was released.\n\nAnswer: 8 TV shows.\n\nExample 4:\n\n"

test = []

for q in tqdm(test_set):
    if USE_STATIC_PROMPT:
        prompt = STATIC_PROMPT
    else:
        if RANDOM_PROMPT:
            hits = random.sample(dev, k=KSHOT)
        else:
            item_passage = ""
            for i,c in enumerate(q['context']):
                item_passage+= "Document {0}: {1}\n\n".format(i+1, c['text'])
            item_passage += "{0}".format(q['question'])
            
            item_embedding = model.encode(item_passage)

            all_top = util.dot_score(item_embedding, passages_embeddings)[0].topk(KSHOT)
            if len(all_top.indices) == 1:
                hits = np.array(dev)[[all_top.indices]].tolist()
            else:
                hits = np.array(dev)[all_top.indices].tolist()

            hits.reverse()
        
        prompt = "For each example, use the documents to create an \"Answer\" and an \"Evidence\" to the \"Question\". Answer \"not enough information\" when not enough information is provided in the documents.\n\n"
        for i, hit in enumerate(hits):
            prompt += "Example {0}:\n\n".format(i+1)
            for j,c in enumerate(hit['context']):
                text = ""
                if c['passage'] == "main":
                    text = "Title: {0}. Content: {1}".format(hit['title'],c['text'])
                else:
                    text = "Title: {0}. Content: {1}".format(c['passage'],c['text'])
                prompt+= "Document {0}: {1}\n\n".format(j+1, text)
            answer = ""
            if hit['answer']['type'] == "span":
                answer = ", ".join([a['text'] for a in hit['answer']["answer_spans"]])
            elif hit['answer']['type'] == "value":
                answer = "{0} {1}".format(hit['answer']['answer_value'],hit['answer']['answer_unit'])
            elif hit['answer']['type'] == "binary":
                answer = hit['answer']['answer_value']
            elif hit['answer']['type'] == "none":
                answer = "Not enough information."
            prompt += "Question: Based on the above documents, {0}\n\nEvidence: {1}\n\nAnswer: {2}.\n\n".format(hit['question'], hit['explanation'].replace('\n',''), answer)
        
        prompt += "Example {0}:\n\n".format(i+2)

    
    limit_per_query = 3
    min_total = 3
    chosen = []
    if len(q['decomposition']) < 2:
        chosen = q['decomposition'][0]['documents'][:min_total]
    else:
        for d in q['decomposition']:
            chosen = chosen +d['documents'][:limit_per_query]
    for i, c in enumerate(chosen):
        text = ""
        text = "Title: {0}. Content: {1}".format(c['title'], c['text'])
        prompt+= "Document {0}: {1}\n\n".format(i+1, text)
    
    prompt += "Question: Based on the above documents, {0}\n\nEvidence:".format(q['question'])
    
    q['prompt'] = prompt
    answers = []
    if q['answer']['type'] == "span":
        at = ", ".join([a['text'] for a in q['answer']["answer_spans"]])
        answers.append(at)
    elif q['answer']['type'] == "value":
        at = "{0} {1}".format(q['answer']['answer_value'],q['answer']['answer_unit'])
        answers.append(at)
    elif q['answer']['type'] == "binary":
        answers.append(q['answer']['answer_value'])
    elif q['answer']['type'] == "none":
        answers.append("Not enough information")
    q['clean_answers'] = answers
    test.append(q)

100%|██████████| 150/150 [00:17<00:00,  8.41it/s]


In [ ]:
from tqdm import tqdm
import json
import re

temperature = 0
attempts = 1
pattern = "(?<=Answer:)(.*)$"

for item in tqdm(test):
    item['responses'] = []
    item['completions'] = []
    for i in range(attempts):
        res = generate(item['prompt'],temperature=temperature)
        if "Answer" not in res:
            item["new_prompt"] = "{0}{1}\n\nAnswer:".format(item["prompt"], res)
            res2= generate(item["new_prompt"])
            item['results'] = "{0}\n\nAnswer: {1}".format(res, res2)
            item['responses'].append(res2)
            item['completions'].append("{0}\n\nAnswer: {1}".format(res, res2))
            item["asked_twice"] = True
        else:
            matches = re.findall(pattern, res)
            if len(matches) > 0: 
                response = matches[0]
                item['responses'].append(response)
            item['results'] = res 
            item['completions'].append(res)
            item["asked_twice"] = False
        
json.dump(test, open("results/iirc.json",'w'))

## Evaluation

In [18]:
import json
import argparse
import collections
import numpy as np
import os
import re
import string
import sys
import unicodedata

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  def remove_accents(input_str):
      nfkd_form = unicodedata.normalize('NFKD', input_str)
      only_ascii = nfkd_form.encode('ASCII', 'ignore')
      return only_ascii.decode("utf-8")

  return white_space_fix(remove_articles(remove_punc(lower(remove_accents(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1


In [ ]:
from collections import Counter
from tqdm import tqdm

f1s = []
ems = []

for item in tqdm(test):
    normalised = [normalize_answer(a.replace('\n','')) for a in item['responses']]
    c = Counter(normalised)
    response = c.most_common(1)[0][0]
    if "Not enough information provided in the documents." == item['clean_answers'][0]:
        item['clean_answers'][0] = "Not enough information"
    f1 = compute_f1(item['clean_answers'][0], response)
    f1s.append(f1)
    ems.append(compute_exact(item['clean_answers'][0], response))

print("F1:",np.mean(f1s))
print("EM:",np.mean(ems))